# Langchain 1.0

- langchain 1.0 실습 가이드입니다.
- 0버전과 문법 및 객체 구조가 매우 다르므로 GPT를 통한 코드 작성이 매우 어렵습니다. 반드시 작동이 확인된 예시 코드와 공식 문서를 기반으로 개발하시기 바랍니다.
  - https://docs.langchain.com/oss/python/langchain/quickstart
  - https://reference.langchain.com/python/langgraph/graphs/
- markdown 셀의 header로 목차를 확인하고 코드 흐름을 따라가면서 이해되지 않는 개념이 있으면 함께 드린 마크다운 파일을 다시 읽어보세요!

## 0. 환경 설정

- Python 3.12.12 기반의 conda 가상 환경에서 테스트되었습니다.
- environment.yml 또는 requirements.txt 파일을 통해 환경을 재현하실 수 있습니다.
- 환경 변수만 잘 등록해준다면 venv, colab 등 다른 실행 환경에서도 작동할 것으로 예상되나 버전 정보를 잘 확인해주세요.
- 코드를 위에서부터 순서대로 실행시키지 않을 경우 오류가 발생할 수 있습니다.
- 개별 셀 단위로 실행할 때는 모듈 import, agent 및 model 객체 정의가 의도대로 되어 있는지 확인하세요!

In [93]:
# 공통 라이브러리 import
import os

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 예쁜 출력
from pprint import pprint

## 1. langsmith 연결
- langsmith는 langgraph / langchain Agent의 실행을 기록하고, 시각화 및 분석을 제공하는 Observability 플랫폼입니다.
- 모든 입력을 tracing하여 입출력 확인, 토큰 사용량 및 비용 확인, 에러 로그 시각화, 지연시간 측정 등의 기능을 제공합니다.
- 아래 세 가지 환경변수만 잘 등록해놓는다면, 따로 코드로 구현하지 않아도 Agent 실행에 대한 모든 로그를 langsmith에서 확인할 수 있습니다.
  - LANGSMITH_TRACING
  - LANGSMITH_API_KEY
  - LANGSMITH_PROJECT

## 2. 단일 Model 호출

- Agent 없이 단일 Model을 호출하는 방법을 알아봅니다.
- invoke()는 모델 또는 체인을 한 번 실행하여 모델의 응답을 반환받는 메서드입니다.
- Agent와 체인을 만들기 전에, 단일 Model을 호출해봅시다.

In [ ]:
# OpenAI 채팅 모델 초기화의 가장 기본적인 형태
# init_chat_model은 여러 모델을 초기화할 수 있는 범용적이고 간단한 함수입니다.
# 그러나 모델별 세부 설정을 위해서는 각 모델 벤더들이 제공하는 모듈의 클래스를 활용하는 것이 좋습니다. 이 방법은 뒤에 등장하니, 실제 개발 시에는 그걸 사용하세요!
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-5-nano") # api key는 환경 변수에서 자동으로 로드됩니다. api_key 매개변수로 직접 전달할 수도 있습니다.

In [95]:
# invoke 메서드로 독립적인 LLM API를 호출합니다.
response = model.invoke('안녕')

# 모델이 보내준 응답 전체를 출력하면, 토큰 사용량 등 메타 데이터가 함께 출력됩니다.
pprint(response)

AIMessage(content='안녕! 오늘은 무엇을 도와줄까?\n\n- 간단한 질문에 답하기\n- 한국어 학습 도움(문법, 표현, 발음 연습)\n- 번역이나 의역\n- 글쓰기 아이디어나 초안, 요약\n- 코딩 문제 해결\n- 여행 정보나 음식 레시피\n\n그 외에 하고 싶은 주제가 있다면 편하게 말해줘!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 546, 'prompt_tokens': 8, 'total_tokens': 554, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYmzM36K6pzZHD5lucD0hxZBHYLfn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e213b493-7a57-4c8f-ac13-fcf2530782a5-0', usage_metadata={'input_tokens': 8, 'output_tokens': 546, 'total_tokens': 554, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 448}})


In [96]:
# 모델이 보내준 실제 답변 내용만 출력하려면 응답의 content 속성을 참조합니다.
pprint(response.content)

('안녕! 오늘은 무엇을 도와줄까?\n'
 '\n'
 '- 간단한 질문에 답하기\n'
 '- 한국어 학습 도움(문법, 표현, 발음 연습)\n'
 '- 번역이나 의역\n'
 '- 글쓰기 아이디어나 초안, 요약\n'
 '- 코딩 문제 해결\n'
 '- 여행 정보나 음식 레시피\n'
 '\n'
 '그 외에 하고 싶은 주제가 있다면 편하게 말해줘!')


In [97]:
# 모델 초기화 시 다양한 파라미터를 지정할 수 있습니다.
# OpenAI 모델 외 다양한 모델에 대해서도 동일한 파라미터로 요청을 보낼 수 있습니다.
model = init_chat_model(
    model = "openai:gpt-5-nano",
    temperature=0.8, # 응답의 창의성. GPT-5 계열 모델들은 temperature 파라미터를 지원하지 않지만, Langchain에서는 일관된 인터페이스를 제공하기 때문에 오류가 발생하지 않습니다.
    max_tokens=2000, # 최대 토큰 수
    timeout=20, # 요청 제한 시간(초)
    max_retries=2, # 요청 실패 시 재시도 횟수
    )

In [98]:
# 파라미터가 적용된 모델로도 다시 호출해봅시다.
response = model.invoke('너 웹 서치 기능도 가지고 있니?') # 기본 모델은 웹 서치 기능을 가지고 있지 않습니다.
pprint(response.content)

('현재 기본 모드로는 자동으로 실시간 웹 검색을 실행하지는 않아요. 제 답변은 제 학습 데이터와 지식 범위에 기반합니다. 다만 사용하는 '
 '플랫폼에 웹 검색/브라우징 기능이 있다면 그 기능으로 최신 정보를 찾아드릴 수 있습니다.\n'
 '\n'
 '원하시면 다음을 도와드릴 수 있어요.\n'
 '- 웹 검색 기능이 활성화되어 있으면 제가 실시간 정보를 찾아 요약하고 출처를 제공합니다.\n'
 '- 활성화되지 않았다면, 제가 최신 정보를 찾을 수 있도록 검색어를 함께 구성해 드리거나, 직접 찾아볼 수 있는 방법을 안내해 '
 '드립니다.\n'
 '- 이미 알고 있는 웹 페이지가 있다면 그 링크를 주시면 요약이나 핵심 정리도 가능합니다.\n'
 '\n'
 '어떤 정보를 찾고 계신가요? 주제나 기간을 알려주시면 적합한 검색어를 제안해 드릴게요. 예를 들면:\n'
 '- "최근 AI 규제 동향 한국 2024년 이후"\n'
 '- "맥북 M3 성능 비교"\n'
 '- "한국 금융시장 오늘 시세 요약"')


## 3. Agent 생성

- 아무 추가 기능 없는 단일 모델 호출하려고 Langchain을 쓰는 건 아니죠.
- Langchain의 핵심 기능인 Agent를 만들어보겠습니다.

In [99]:
# Agent 생성에 필요한 모듈입니다.
from langchain.agents import create_agent

# 앞서 초기화한 Model을 사용하여 Agent를 생성합니다.
agent = create_agent(
    model=model, # 사용할 LLM 모델을 미리 초기화(init)한 후 인자로 전달하세요.
    tools=[] # tool은 필수 인자지만, 빈 리스트를 전달하면 아무 tool도 없는 Agent를 생성할 수 있습니다.
    )

In [100]:
# 모델에 전달할 메시지를 OpenAI의 Chat API 형식에 맞게 작성합니다.
# 다른 모델은 다른 형식을 요구할 수 있으니, 모델 문서를 참고하세요.
messages = [
    {"role": "user", "content": "오늘 서울 날씨는 어때?"},
]

# Agent를 통해 모델을 호출합니다. Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
# 검색 tool이 없기 때문에, 실시간 날씨 정보를 제공하지는 못할 것입니다.
response = agent.invoke({"messages":messages})
pprint(response)

{'messages': [HumanMessage(content='오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='62fd436c-10bd-417f-99e3-33fa755c1cf4'),
              AIMessage(content='죄송하지만 지금 실시간 날씨 정보에 접속할 수 없어 오늘의 서울 날씨를 바로 알려드릴 수 없어요. 다만 빠르게 확인하는 방법을 안내해 드릴게요.\n\n- 스마트폰의 기본 날씨 앱 확인\n- 네이버 날씨나 다음 날씨 같은 포털 사이트의 서울 날씨 페이지\n- 기상청 홈페이지에서 서울 지역 일기예보 확인\n\n원하시면 제가 바로 확인하는 방법을 따라가실 수 있도록 한두 가지 링크와 요약 화면 구성 방법도 정리해 드릴게요. 또한 오늘의 온도대에 맞춘 옷차림 팁이나 예상 강수 여부 등에 대한 일반적인 정보도 도움이 필요하신가요? 어떤 방식으로 도와드릴지 알려 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1325, 'prompt_tokens': 14, 'total_tokens': 1339, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1152, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYmzbSUOhDyOdnAVISdohXFEixyy0', 'service_tier'

In [101]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('죄송하지만 지금 실시간 날씨 정보에 접속할 수 없어 오늘의 서울 날씨를 바로 알려드릴 수 없어요. 다만 빠르게 확인하는 방법을 안내해 '
 '드릴게요.\n'
 '\n'
 '- 스마트폰의 기본 날씨 앱 확인\n'
 '- 네이버 날씨나 다음 날씨 같은 포털 사이트의 서울 날씨 페이지\n'
 '- 기상청 홈페이지에서 서울 지역 일기예보 확인\n'
 '\n'
 '원하시면 제가 바로 확인하는 방법을 따라가실 수 있도록 한두 가지 링크와 요약 화면 구성 방법도 정리해 드릴게요. 또한 오늘의 온도대에 '
 '맞춘 옷차림 팁이나 예상 강수 여부 등에 대한 일반적인 정보도 도움이 필요하신가요? 어떤 방식으로 도와드릴지 알려 주세요.')


In [102]:
# Agent 생성 시에도 다양한 파라미터를 지정할 수 있습니다.
# 오늘 다 다뤄보지는 못하겠지만, Agent의 세부 동작은 모두 이 파라미터들로 조정할 수 있습니다.
# 아래 있는 것들 외에도 더 많은 파라미터가 있으니 공식 문서를 참고하세요.
agent = create_agent(
    model=model,
    tools=[], # tool 리스트는 다음 챕터에서 다뤄보겠습니다.
    system_prompt="You are a helpful assistant.", # Agent의 시스템 프롬프트입니다.
    middleware=[], # Agent의 동작 사이에 발생하는 요청이나 응답을 가로채서 처리하는 고급 기능입니다.
)

## 4. tool 정의

- 위에서 만든 Agent는 별다른 기능이 없으니 단일 모델 호출과 별로 다를 게 없죠.
- 이제 Agent가 사용할 tool을 만들어 봅시다.
- tool의 종류를 세 가지로 나누어 각각 만들어보겠습니다.

### 4-1. 커스텀 tool 정의

- 직접 만든 Python 함수를 tool로 사용할 수 있습니다.
- URL 또는 라이브러리를 통해 API를 호출하여 응답을 반환하는 tool도 이 방법으로 구현할 수 있습니다.
- tool을 만들 때 typehint와 docstring을 잘 작성해주는 것이 매우 중요합니다! 에이전트가 도구를 선택할 때 참조합니다.

In [103]:
# tool 정의를 위해 필요한 모듈입니다.
from langchain.tools import tool

# 커스텀 tool 생성
@tool # 데코레이터를 사용하여 tool로 등록합니다.
def calculator(num_1:int, num_2:int) -> int: # typehint는 Agent가 tool의 입출력 형식을 이해하는 데 도움을 줍니다. 안정적인 작동을 위해 반드시 작성하는게 좋습니다.
    """입력받은 두 수의 덧셈을 반환합니다.""" # docstring은 tool의 설명으로 사용됩니다. Agent가 tool을 선택하는 데 도움을 줍니다.
    return num_1 + num_2

### 4-2. langchain-community에 통합된 외부 tool 사용

- 라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있습니다.
- 웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있으니 목록을 확인하고 활용해봅시다.
- https://docs.langchain.com/oss/python/integrations/providers/overview

In [104]:
# langchain community에서 통합되어 있는 외부 tool의 목록을 확인해봅시다.
import importlib, pkgutil

package = importlib.import_module("langchain_community.tools")

for mod in pkgutil.iter_modules(package.__path__):
    print(mod.name)

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [105]:
# duckduckgo는 api key 없이도 사용할 수 있는 웹 검색 도구입니다.
import langchain_community.tools.ddg_search

# duckduckgo 검색 tool의 속성을 확인하여 검색 기능 호출 방법을 알아봅시다.
print(dir(langchain_community.tools.ddg_search))

['DuckDuckGoSearchRun', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'tool']


In [106]:
# 위에서 확인한 duckduckgo의 검색 기능을 import합니다.
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# 검색 tool 인스턴스를 생성합니다.
ddg_search_tool = DuckDuckGoSearchRun()

In [107]:
# 세 번째 방법을 배우기 전에, 위에서 만든 두 가지 tool을 Agent에게 전달해봅시다.
# nano 모델은 Agent로 사용하기엔 너무 성능이 낮아 무한 루프 등 의도하지 않은 동작이 발생할 수 있으므로 mini 모델로 변경하겠습니다.
model = init_chat_model("openai:gpt-5-mini")

# agent 생성 및 tool 전달
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
)

In [108]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?"),
]

# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke({"messages":messages})


# Agent가 보내준 전체 응답에는 추론 과정과 tool 사용 내역이 포함되어 있습니다.
pprint(response)

{'messages': [HumanMessage(content='2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='7c89a7b8-d2e0-4f29-8066-bf4851cdde2d'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 207, 'total_tokens': 428, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYmzk4w5zyoHmLczWsTqQT8Bh0fhj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--7e4614bd-7538-46e2-ad18-0ad10938bd11-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '서울 날씨 오늘'}, 'id': 'call_TJbqkIVMvntINp0KNR0a5U4k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 207, 'output_toke

In [109]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('2 더하기 3은 5입니다.\n'
 '\n'
 '오늘 서울 날씨는(검색 결과 기준) 아침에 짙은 안개로 쌀쌀하고, 낮에는 포근해 낮 최고 기온이 대략 20–22°C 정도로 예보되어 '
 '있습니다. 다만 저는 실시간 관측소에 직접 연결되어 있지 않아 현재 기온·강수 여부·미세먼지 등 세부 정보는 바로 확인해 드리기 '
 '어렵습니다. 원하시면 지금 정확한 현재 기온·시간별 예보·미세먼지 등을 바로 찾아서 알려드릴까요?')


### 4-3. 모델 vendor들이 제공하는 tool을 모델에 바인딩

#### **※ 주의!! ※**
- 현재 버전에서 Agent를 통하지 않고 단일 Model을 직접 호출해야만 사용할 수 있는 기능입니다.
- Agent를 활용하는 경우 사용할 수 없는 방법이므로 헷갈리지 마세요!
- 설계가 복잡해지고 세밀한 구조 설계, 유지보수를 어렵게 만드는 방식이니 사용하지 마세요.
- 다만, 이 방법을 아예 모르고 있으면 GPT나 Docs 보고 코드 쓰다가 tool 정의 및 호출 방법을 섞어서 쓰다가 코드가 꼬일 수 있기 때문에 개념만 이해하고 넘어갑시다.
---
#### 실습
- LLM API를 제공하는 vendor들은 모델에서 곧바로 이용할 수 있는 tool을 함께 서비스하고 있습니다.
- 예를 들어, OpenAI의 경우 웹 서치, MCP 서버 연결, 벡터 스토어 검색, 코드 실행 등의 도구를 제공합니다.
- vendor가 제공하는 tool을 Agent에 바인딩하여 사용해보겠습니다.

In [110]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.5,
    model_name="gpt-5-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울특별시(서울)입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 15, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYn07e7EYdAtkzLHmJUMmITUrsY32', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--5f18f1b3-bd1f-437d-bacb-8dea1715d81d-0' usage_metadata={'input_tokens': 15, 'output_tokens': 151, 'total_tokens': 166, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}}


In [111]:
# 모델 vendor들이 제공하는 tool을 모델에 바인딩하기 위해 필요한 모듈입니다.
from langchain_openai import ChatOpenAI

# OpenAI 전용 Model 초기화 메서드입니다.
# OpenAI 모델에서 제공하는 tool을 바인딩하기 위해서는 모델 초기화시 ChatOpenAI 클래스를 사용해야 합니다.
model = ChatOpenAI(
    model="gpt-5-mini",
)

# ChatOpenAI 모델 인스턴스에 커스텀 tool을 바인딩합니다.


# 모델이 사용할 tool 리스트를 정의합니다.
# 딕셔너리 형태의 web search tool은 openai가 제공하는 tool이고, calculator는 위에서 직접 만든 커스텀 tool입니다.
# ddg는 웹 검색 역할이 중복되므로 제외합니다.
tools = [{"type": "web_search"}, calculator]

model_with_calculator = model.bind_tools(tools) 

# model을 호출합니다.
response = model_with_calculator.invoke(
    "올해 11월에 한국에서 개봉하는 주요 영화 3개만 알려줘. 그리고 12311 더하기 112455는 몇이야?",
)

# 추론 과정에서 커스텀 tool이 호출되면, 추론 루프를 멈추고 AIMessage를 반환합니다.
# Agent가 아닌 단일 모델에 대한 invoke이기 때문에 커스텀 tool 호출에는 대응하지 못하기 때문입니다.
# 별도로 다시 invoke를 체이닝하는 등 추가 코드가 필요하나, 너무 구린 방식이니까 그냥 넘어가겠습니다.
pprint(response)

AIMessage(content=[{'id': 'rs_0437b43581507ac900690c3480f78081928bc741b5650ab945', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0437b43581507ac900690c348284c48192bc16ce416ba34b2b', 'action': {'query': 'South Korea November 2025 movie releases November 2025 release schedule Korea major films', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0437b43581507ac900690c348514148192af14e80d3c98d748', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0437b43581507ac900690c34858ba08192aa755889d59189b8', 'action': {'query': "2025년 11월 개봉 영화 한국 11월 개봉작 2025 '개봉' '11월' '한국' '영화' '2025' ", 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0437b43581507ac900690c34870cc88192adac08bcb9049605', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0437b43581507ac900690c34892d5c8192b0eff089791945f7', 'action': {'query': "위키드: 포 굿 2025 한국 개봉일 11월 19일 CGV '위키드 2' '개봉' '한국' '11월' '2025' ", 'type': 'search'}, 'status': 'completed', 'type': 'web_search

## 5. 메모리

- 맥락을 유지하고 개인화된 응답을 제공하기 위해 checkpointer 객체에 메모리를 저장합니다.
- 스레드 내에서만 유지할 메모리와 DB에 저장할 메모리를 구분하여 메모리 객체를 생성해보겠습니다.

#### 5-1. 단기 메모리

- 세션이 유지되는 동안 대화 기록을 저장해보겠습니다.
- 세션이 종료되면 이 기억은 사라지며, 복구할 수 없습니다.

In [112]:
# 쓰레드 내 단기 기억 저장용 객체를 생성하기 위한 모듈입니다.
from langgraph.checkpoint.memory import InMemorySaver

# InMemorySaver 객체를 포함하여 Agent 생성
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
    checkpointer=InMemorySaver(),
)

In [113]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="한국에서 가볼 만한 여행지 추천해줘."),
]

In [114]:
# Agent 호출
# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke(
    {"messages":messages},
    # 설정값으로 대화 기록을 저장할 쓰레드 번호를 함께 넘겨줘야 합니다.
    # 대화 기록이 들어 있는 쓰레드는 세션 종료시 삭제됩니다.
    {"configurable": {"thread_id": "1"}},
    )

pprint(response)

{'messages': [HumanMessage(content='한국에서 가볼 만한 여행지 추천해줘.', additional_kwargs={}, response_metadata={}, id='e0f94761-a122-4d8a-a07c-2eb307df4b2b'),
              AIMessage(content='좋아요 — 여행 스타일이나 기간을 알려주시면 더 맞춤형으로 추천해드릴게요. 우선 인기 있고 매력적인 한국 여행지를 목적별·기간별로 간단히 정리해 드립니다.\n\n추천 여행지(하이라이트 + 권장 일정)\n- 서울 — 경복궁·북촌·인사동(전통), 홍대·이태원·강남(젊음·쇼핑·밤문화), 남산·한강(전경/산책). 1~3일.\n- 부산 — 해운대·광안리(해변), 감천문화마을(포토스팟), 자갈치시장(해산물). 1~2일(주말 추천).\n- 제주도 — 한라산 등반, 성산일출봉, 만장굴, 협재·함덕 해변, 올레길 걷기. 3~5일.\n- 경주 — 불국사·석굴암·첨성대·안압지 등 신라 역사 투어. 1~2일.\n- 전주 — 전주한옥마을, 한옥 숙박, 비빔밥·전통한식 맛집. 1일(미식·문화).\n- 강원도(설악산/강릉/속초) — 설악산 단풍/등산, 동해 해변, 커피거리(강릉). 1~3일(계절 활동 다양).\n- 안동(하회마을) — 전통 민속마을과 문화체험. 1일.\n- 보성·남해·순천 — 보성 녹차밭, 순천만 습지, 남해의 풍경·드라이브 코스. 1~2일.\n- 담양 — 메타세쿼이아 길, 죽녹원(여유로운 자연). 1일.\n- DMZ(비무장지대) 투어 — 역사·안보 관람(사전 예약 필요). 반나절~1일.\n- 평창·강릉(겨울스포츠) — 스키·스노보드(겨울). 1~3일.\n- 부산 근교/경남(거제·통영) — 한려수도 풍경, 섬 투어, 해산물. 1~2일.\n\n테마별 추천\n- 자연·트레킹: 제주도(한라산), 설악산, 지리산\n- 해변·휴양: 제주, 부산(해운대), 강릉\n- 역사·문화: 경주, 전주, 안동\n- 음식여행: 광주·전주(전통 한식), 부산(해산물), 서울(다양

In [115]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('좋아요 — 여행 스타일이나 기간을 알려주시면 더 맞춤형으로 추천해드릴게요. 우선 인기 있고 매력적인 한국 여행지를 목적별·기간별로 간단히 '
 '정리해 드립니다.\n'
 '\n'
 '추천 여행지(하이라이트 + 권장 일정)\n'
 '- 서울 — 경복궁·북촌·인사동(전통), 홍대·이태원·강남(젊음·쇼핑·밤문화), 남산·한강(전경/산책). 1~3일.\n'
 '- 부산 — 해운대·광안리(해변), 감천문화마을(포토스팟), 자갈치시장(해산물). 1~2일(주말 추천).\n'
 '- 제주도 — 한라산 등반, 성산일출봉, 만장굴, 협재·함덕 해변, 올레길 걷기. 3~5일.\n'
 '- 경주 — 불국사·석굴암·첨성대·안압지 등 신라 역사 투어. 1~2일.\n'
 '- 전주 — 전주한옥마을, 한옥 숙박, 비빔밥·전통한식 맛집. 1일(미식·문화).\n'
 '- 강원도(설악산/강릉/속초) — 설악산 단풍/등산, 동해 해변, 커피거리(강릉). 1~3일(계절 활동 다양).\n'
 '- 안동(하회마을) — 전통 민속마을과 문화체험. 1일.\n'
 '- 보성·남해·순천 — 보성 녹차밭, 순천만 습지, 남해의 풍경·드라이브 코스. 1~2일.\n'
 '- 담양 — 메타세쿼이아 길, 죽녹원(여유로운 자연). 1일.\n'
 '- DMZ(비무장지대) 투어 — 역사·안보 관람(사전 예약 필요). 반나절~1일.\n'
 '- 평창·강릉(겨울스포츠) — 스키·스노보드(겨울). 1~3일.\n'
 '- 부산 근교/경남(거제·통영) — 한려수도 풍경, 섬 투어, 해산물. 1~2일.\n'
 '\n'
 '테마별 추천\n'
 '- 자연·트레킹: 제주도(한라산), 설악산, 지리산\n'
 '- 해변·휴양: 제주, 부산(해운대), 강릉\n'
 '- 역사·문화: 경주, 전주, 안동\n'
 '- 음식여행: 광주·전주(전통 한식), 부산(해산물), 서울(다양한 전문 맛집)\n'
 '- 사진·감성: 감천문화마을, 북촌, 보성녹차밭, 남해\n'
 '\n'
 '이동 팁\n'
 '- 장거리: KTX(서울↔부산/경주/전주 등) 추천

In [116]:
# LLM이 이전 대화 맥락을 잘 기억하는지 확인해봅시다.

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="서울 안에서는?"),
]

response = agent.invoke(
    {"messages":messages},
    # 설정값으로 대화 기록을 저장할 쓰레드 번호를 함께 넘겨줘야 합니다.
    # 대화 기록이 들어 있는 쓰레드는 세션 종료시 삭제됩니다.
    {"configurable": {"thread_id": "1"}},
    )

pprint(response)

{'messages': [HumanMessage(content='한국에서 가볼 만한 여행지 추천해줘.', additional_kwargs={}, response_metadata={}, id='e0f94761-a122-4d8a-a07c-2eb307df4b2b'),
              AIMessage(content='좋아요 — 여행 스타일이나 기간을 알려주시면 더 맞춤형으로 추천해드릴게요. 우선 인기 있고 매력적인 한국 여행지를 목적별·기간별로 간단히 정리해 드립니다.\n\n추천 여행지(하이라이트 + 권장 일정)\n- 서울 — 경복궁·북촌·인사동(전통), 홍대·이태원·강남(젊음·쇼핑·밤문화), 남산·한강(전경/산책). 1~3일.\n- 부산 — 해운대·광안리(해변), 감천문화마을(포토스팟), 자갈치시장(해산물). 1~2일(주말 추천).\n- 제주도 — 한라산 등반, 성산일출봉, 만장굴, 협재·함덕 해변, 올레길 걷기. 3~5일.\n- 경주 — 불국사·석굴암·첨성대·안압지 등 신라 역사 투어. 1~2일.\n- 전주 — 전주한옥마을, 한옥 숙박, 비빔밥·전통한식 맛집. 1일(미식·문화).\n- 강원도(설악산/강릉/속초) — 설악산 단풍/등산, 동해 해변, 커피거리(강릉). 1~3일(계절 활동 다양).\n- 안동(하회마을) — 전통 민속마을과 문화체험. 1일.\n- 보성·남해·순천 — 보성 녹차밭, 순천만 습지, 남해의 풍경·드라이브 코스. 1~2일.\n- 담양 — 메타세쿼이아 길, 죽녹원(여유로운 자연). 1일.\n- DMZ(비무장지대) 투어 — 역사·안보 관람(사전 예약 필요). 반나절~1일.\n- 평창·강릉(겨울스포츠) — 스키·스노보드(겨울). 1~3일.\n- 부산 근교/경남(거제·통영) — 한려수도 풍경, 섬 투어, 해산물. 1~2일.\n\n테마별 추천\n- 자연·트레킹: 제주도(한라산), 설악산, 지리산\n- 해변·휴양: 제주, 부산(해운대), 강릉\n- 역사·문화: 경주, 전주, 안동\n- 음식여행: 광주·전주(전통 한식), 부산(해산물), 서울(다양

In [117]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('좋아요 — 서울 안에서 할 만한 것들을 목적별·지역별로 정리해드릴게요. 원하시면 이 중에서 일정(하루/이틀)로 바로 짜드릴게요.\n'
 '\n'
 '주요 지역별 하이라이트\n'
 '- 경복궁·북촌·인사동(종로)\n'
 '  - 전통 궁궐(경복궁, 창덕궁+후원 예약 권장), 한옥 골목(북촌), 전통차·공예(인사동).\n'
 '  - 팁: 한복 착용 시 주요 궁궐 무료 입장 가능(체크 필요).\n'
 '- 명동·남대문·명동성당\n'
 '  - 화장품·패션 쇼핑, 거리 음식, 외국인 관광객 많은 지역.\n'
 '- 동대문·DDP\n'
 '  - 패션 도매상가, 밤샘 쇼핑, 디자인 전시(DDP).\n'
 '- 홍대·합정·연남\n'
 '  - 젊음의 거리, 인디음악·카페·클럽, 스트리트 퍼포먼스, 개성 있는 식당·플리마켓.\n'
 '- 이태원·한남·경리단길\n'
 '  - 글로벌 음식, 바·나이트라이프, 외국 문화가 공존하는 동네.\n'
 '- 강남·청담·압구정\n'
 '  - 럭셔리 쇼핑, 트렌디 카페·레스토랑, 코엑스(쇼핑몰·아쿠아리움·별마당도서관).\n'
 '- 성수·왕십리\n'
 '  - 카페·디자인 스튜디오·공장 리노베이션 스폿(성수동 수제화 거리).\n'
 '- 잠실·석촌호수·롯데월드타워\n'
 '  - 전망대, 쇼핑몰, 롯데월드(실내·실외 놀이공원), 석촌호수 산책.\n'
 '- 한강 공원(여의도·반포·잠원 등)\n'
 '  - 자전거·피크닉·유람선·야경(반포대교 분수).\n'
 '- 광장시장·남대문·노량진·노포 골목\n'
 '  - 재래시장·전통 먹거리(빈대떡·육회·회·해산물).\n'
 '\n'
 '테마별 추천 활동\n'
 '- 역사·전통: 경복궁(수문장 교대), 창덕궁 후원, 국립민속박물관, 북촌 산책\n'
 '- 예술·박물관: 국립중앙박물관, MMCA(국립현대미술관 서울관), 리움미술관, DDP 전시\n'
 '- 쇼핑·트렌드: 명동·동대문·가로수길·코엑스·성수\n'
 '- 먹방·재래시장: 광장시장(마약김밥, 빈대떡), 통인시장(도시락카페), 노량진

### 5-2. 장기 메모리

- 위에서 InMemorySaver로 만든 기억은 세션이 사라지면 함께 없어진다고 했습니다.
- 사용자가 앱을 껐다 켜고, 기존에 대화를 나눴던 세션에 들어왔을 때도 기억이 남아있으려면, 대화 기록을 영속적으로 저장해야 합니다.
- sqlite DB에 대화 기록을 저장할 수 있는 Agent를 만들어보겠습니다.
- 다른 DB를 사용할 경우 해당 DB 관련 패키지 install 및 import가 필요합니다.

In [118]:
# sqlite용 checkpointer 객체 생성을 위한 모듈입니다.
from langgraph.checkpoint.sqlite import SqliteSaver

# checkpointer 객체 생성
checkpointer = SqliteSaver.from_conn_string("checkpoints.db")

# DB는 with문 안에서만 연결되어 있습니다.
with SqliteSaver.from_conn_string("checkpoints.db") as checkpointer:
    # 테이블이 없을 경우 생성
    checkpointer.setup()
    # Agent 객체 생성
    agent = create_agent(
        model=model,
        tools=tools,
        checkpointer=checkpointer,
    )
    # 실행 후, sqlite DB에 대화 기록이 잘 저장되었는지 확인해보세요.
    # Agent에 전달할 메시지 객체를 생성합니다.
    messages = [
        HumanMessage(content="langchain의 설계 철학에 대해 알려줘."),
    ]

    # Agent invoke
    response = agent.invoke(
        {"messages":messages},
        {"configurable": {"thread_id":1}},
        )

    # 출력
    pprint(response)


{'messages': [HumanMessage(content='서울 안에서는?', additional_kwargs={}, response_metadata={}, id='56162ef8-f25f-4675-932a-9bf25b330738'),
              AIMessage(content=[{'id': 'rs_0fc6438b81b8efa700690ad71452d8819e8ba8111b5c2caa0c', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '무엇에 대해 여쭤보신 건가요—좀 더 구체적으로 알려주실 수 있나요?\n\n가능한 의미 예시:\n- 서울 안에서는 어디를 가볼까요? (관광·맛집 추천)\n- 서울 안에서는 대중교통/택시 이용이 어떤가요?\n- 서울 안에서는 배달/택배가 되나요?\n- 서울 안에서는 코로나/방역 규칙은 어떤가요?\n- 서울 안에서는 집값·월세는 어느 정도인가요?\n- 서울 안에서는 주차 규정(혹은 자전거 관련) 어떻게 되나요?\n\n원하시는 항목이나 상세 조건(여행일정, 예산, 출발지 등)을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad719012c819eabfeea13ab32813c'}], additional_kwargs={}, response_metadata={'id': 'resp_0fc6438b81b8efa700690ad713b084819e89a7449b838045ee', 'created_at': 1762318099.0, 'metadata': {}, 'model': 'gpt-5-mini-2025-08-07', 'object': 'response', 'service_tier': 'default', 'status': 'completed', 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07'}, id='resp_0f

In [119]:
# 저장된 데이터를 조회해보겠습니다.
from langgraph.checkpoint.sqlite import SqliteSaver

# DB를 조회할 때도, with문 아래에서 checkpointer를 호출해야 합니다.
with SqliteSaver.from_conn_string("checkpoints.db") as saver:
    checkpoint_tuple = saver.get_tuple({"configurable": {"thread_id": "1"}})

    # 이하는 db에 저장된 객체에서 사용자와 AI의 대화 부분만 파싱하여 출력하기 위한 코드입니다.
    # 불필요하게 복잡하고 보기에도 불편하므로, langsmith key를 연결해놓고 거기서 확인하는게 좋습니다.
    checkpoint_data = checkpoint_tuple.checkpoint
    messages = checkpoint_data["channel_values"]["messages"]

    for i, msg in enumerate(messages, 1):
        msg_type = getattr(msg, "type", msg.__class__.__name__)
        print(f"{i:02d}. [{msg_type.upper()}] {msg.content}\n")


01. [HUMAN] 서울 안에서는?

02. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad71452d8819e8ba8111b5c2caa0c', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '무엇에 대해 여쭤보신 건가요—좀 더 구체적으로 알려주실 수 있나요?\n\n가능한 의미 예시:\n- 서울 안에서는 어디를 가볼까요? (관광·맛집 추천)\n- 서울 안에서는 대중교통/택시 이용이 어떤가요?\n- 서울 안에서는 배달/택배가 되나요?\n- 서울 안에서는 코로나/방역 규칙은 어떤가요?\n- 서울 안에서는 집값·월세는 어느 정도인가요?\n- 서울 안에서는 주차 규정(혹은 자전거 관련) 어떻게 되나요?\n\n원하시는 항목이나 상세 조건(여행일정, 예산, 출발지 등)을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad719012c819eabfeea13ab32813c'}]

03. [HUMAN] 안녕?

04. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad8f9809c819e8f43be7e22c372cd', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': '안녕하세요! 반가워요 — 무엇을 도와드릴까요? 이전에 쓰신 “서울 안에서는?” 관련해서\n- 관광·맛집 추천\n- 교통(지하철·버스·택시)\n- 집값·월세 정보\n- 배달·편의 서비스\n- 방역·행사·날씨 등\n\n원하시는 항목과 날짜·예산·취향 같은 조건을 알려주시면 바로 도와드릴게요.', 'annotations': [], 'id': 'msg_0fc6438b81b8efa700690ad8fce444819eb74f84e66cc7bd83'}]

05. [HUMAN] 안녕?

06. [AI] [{'id': 'rs_0fc6438b81b8efa700690ad97c341c

### ※ 참고
- 단기 메모리든 장기 메모리든, 모델은 쌓여 있는 모든 대화기록을 input받아 대답합니다.
- 따라서 세션 내 대화가 길어질수록 token 사용량이 급격히 늘어나고, 모델이 한 번에 처리할 수 없는 양이 되면 오류를 발생시킵니다.
- 오래된 메시지를 제거하거나 요약하는 등의 방법으로 제어해야 하는데, 마지막 챕터인 미들웨어에서 간단하게 다뤄보겠습니다.
- 서비스 개발시 반드시 공식문서를 참조하여 구현해보시기 바랍니다.
  - https://docs.langchain.com/oss/python/langchain/short-term-memory

## 6. 스트리밍

- Agent의 반복적인 추론으로 인한 지연 시간은 UX를 크게 저하시킵니다.
- 출력을 점진적으로 표시하는 스트리밍 기능을 통해 보완해봅시다.
- OpenAI API는 스트리밍 기능 사용을 위해 기관 인증을 요구하므로 지금은 사용할 수 없습니다.
- Ollama 로컬 LLM을 활용하여 맛만 봅시다!

In [120]:
from langchain_community.chat_models import ChatOllama

# Ollama 데스크탑 앱을 설치하고, 터미널에서 사용할 모델(gemma3)를 미리 다운받아야 합니다.
# ollama pull gemma3

# 모델 지정
model = ChatOllama(
    model="gemma3",
    streaming=True,
)

# Agent 생성 및 tool 전달
agent = create_agent(
    model=model,
)

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="Who is the best football player in history?"),
]

In [122]:
# 스트리밍 출력을 확인하기 위해 내용을 파일에 저장하겠습니다.
# ollama 실행 및 gemma3 모델 pull이 되어있지 않으면 여기서 오류가 발생합니다.
with open("stream_output.txt", "w", encoding="utf-8") as f:
    # agent.stream()으로 호출
    for token, metadata in agent.stream(
        {"messages": messages}, # HumanMessage 객체 전달
        stream_mode="messages", # 스트림으로 받을 단위 설정
    ):
        # 노트북 셀 출력
        print(f"node: {metadata['langgraph_node']}")
        print(f"content: {token.content_blocks}")
        
        # 파일에 저장
        f.write(f"node: {metadata['langgraph_node']}, ")
        f.write(f"content: {token.content_blocks}")
        f.write('\n')
        f.flush()

node: model
content: [{'type': 'text', 'text': 'Okay'}]
node: model
content: [{'type': 'text', 'text': ','}]
node: model
content: [{'type': 'text', 'text': ' this'}]
node: model
content: [{'type': 'text', 'text': ' is'}]
node: model
content: [{'type': 'text', 'text': ' *'}]
node: model
content: [{'type': 'text', 'text': 'the'}]
node: model
content: [{'type': 'text', 'text': '*'}]
node: model
content: [{'type': 'text', 'text': ' question'}]
node: model
content: [{'type': 'text', 'text': ' that'}]
node: model
content: [{'type': 'text', 'text': ' sparks'}]
node: model
content: [{'type': 'text', 'text': ' endless'}]
node: model
content: [{'type': 'text', 'text': ' debate'}]
node: model
content: [{'type': 'text', 'text': ' in'}]
node: model
content: [{'type': 'text', 'text': ' football'}]
node: model
content: [{'type': 'text', 'text': ' ('}]
node: model
content: [{'type': 'text', 'text': 'soccer'}]
node: model
content: [{'type': 'text', 'text': ')'}]
node: model
content: [{'type': 'text', '

## 7. 미들웨어

- 미들웨어를 통해 에이전트의 추론 과정 중간에 개입하여 내부 동작을 커스터마이징할 수 있습니다.
- Agent를 세부적으로 커스터마이징하기 위한 대부분의 작업이 미들웨어를 통합니다.
  - 대화 기록 요약
  - 동작 중 사용자 입력 대기
  - 특정 모델 또는 tool에 대한 호출 제약
  - fallback
  - PII(개인식별정보) 처리 등
- 몇 가지만 간단하게 구현해보겠습니다. 서비스 개발시 공식 문서를 참조하여 다른 기능들도 적용해보세요!
  - https://docs.langchain.com/oss/python/langchain/middleware

### 7-1. 대화 기록 요약
- 대화가 지속될 때 context가 무한히 늘어나지 않도록 이전 대화 기록을 요약해서 전달해봅시다.
- 작업의 특성에 따라 성능을 크게 하락시킬 수 있으니 적용 시 충분한 테스트가 필요합니다.

In [123]:
# 미들웨어에서 대화 기록을 요약하기 위해 사용되는 모듈입니다.
from langchain.agents.middleware import SummarizationMiddleware
# 요약하려면 기록이 있어야 하니까, 단기 메모리를 만들어 줍니다.
from langgraph.checkpoint.memory import InMemorySaver

# model 정의
model = init_chat_model(
    model="openai:gpt-5-mini",
    max_tokens=8000,
    )

# 사용할 middleware 객체를 미리 정의합니다.
middleware_summarize = SummarizationMiddleware(
            model=model, # 요약하는 데에 쓸 모델. 물론 Agent 모델과 다른 걸 써도 됨
            max_tokens_before_summary=1000, # 모델이 처리해야 할 토큰 수가 임계치를 넘으면, 요약을 먼저 수행합니다. 추론이 동작하는지 확인하기 위해 매우 작은 값으로 설정했습니다.
            messages_to_keep=1, # 요약에 포함시키지 않을 최근 메시지의 수를 결정합니다. 너무 낮게 설정하면 최근 대화에 대한 맥락이 소실되니 주의하세요.
            # summary_prompt="요약할 때 이렇게 저렇게 해줘.. 커스터마이징 프롬프트 사용 가능"
        )


# Agent 생성
agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(), # 단기 메모리를 저장할 checkpointer 객체 지정
    middleware=[middleware_summarize, ], # 위에서 정의한 middleware를 Agent에게 전달합니다.
)

In [124]:
# Agent가 요약을 실행하고 있는지 응답을 확인해봅시다.

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="뮤지컬 Wicked의 내용을 Elphaba 입장에서 서술해줘. 그리고 그 내용을 기반으로 Elphaba를 연기할 때 신경써야 할 포인트들을 짚어줘."),
]

# Agent 호출
response = agent.invoke(
    {"messages":messages},
    {"configurable": {"thread_id": "1"}}, # 단기 메모리를 저장할 쓰레드 번호를 전달합니다.
    )

# Agent 호출을 반복하여 대화 기록을 쌓아 봅시다.
messages = [
    HumanMessage(content="Glinda를 어떤 감정으로 바라보는게 좋을까?"),
]

response = agent.invoke(
    {"messages":messages},
    {"configurable": {"thread_id": "1"}}, 
    )

# 세 번째 호출!
messages = [
    HumanMessage(content="도와줘서 고마워. 연기 잘 해볼게."),
]

response = agent.invoke(
    {"messages":messages},
    {"configurable": {"thread_id": "1"}}, 
    )

# 마지막 호출에 대한 응답만 확인해보겠습니다.
# langsmith로 가서, middleware가 어떻게 호출되었는지 꼭 확인해보세요!
pprint(response)

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\n(Elphaba 1인칭 요약)\n- 초록 피부로 태어나 ‘다르다’고 낙인찍힌 이상주의자. 아버지 부재·가정 갈등·언니와의 대비로 외로움과 정의감 형성.\n- 아카데미에서 글린다를 만나 우정이 생기지만 오해와 충돌을 겪음. 마법사와 체제의 위선을 발견하고 진실을 말하려다 추격당함.\n- 피에로와의 로맨스에서 위로와 혼란을 경험. 결국 공개적으로 체제에 맞서며 사람들에게 ‘위키드’로 낙인찍힘.\n- 핵심 주제: ‘다름’으로 인한 고통, 타인을 구하려는 희생, 글린다와의 복합적 연결이 만드는 변화.\n\n연기 포인트(핵심)\n1) 내적 동기와 목표\n- 중심 목표: 인정받고 세상을 바꾸고자 하는 열망을 모든 장면에서 보이게.\n- 장면별 작은 목표(이해받기, 화해, 보호, 폭로 등)를 분명히 설정.\n- 에너지 흐름: 이상→희망→좌절·분노→결의·희생.\n\n2) 감정의 아크\n- 초반: 소외·신중하지만 눈빛엔 불꽃.\n- 중반: 희망과 좌절 교차, 분노·씁쓸함 상승.\n- 후반: 확고한 결의와 외로움, 강하고 넓은 표현 필요.\n\n3) 신체성\n- 초반: 어깨 닫힘, 서투른 걸음, 긴장된 손동작.\n- 친밀해질수록 몸이 풀림(어색한 미소 포함).\n- 정체성 확립 시: 가슴을 열고 시선·자세 상승.\n- ‘다름’ 표시에 작은 신체 반응(눈썹·한숨·피부 만짐) 활용.\n\n4) 표정·눈빛\n- 초록 메이크업으로 세부 표정 묻힘 → 눈과 입 표현을 더 크고 명확하게.\n- 분노: 좁고 강한 눈빛. 상처: 크게 뜬 눈과 물기. 결의: 고정된 시선.\n- 진심의 웃음은 눈부터, 위선은 입부터.\n\n5) 보컬(노래별)\n- 전반: 안정된 브레스, 체스트↔믹스 전환 자연스럽게.\n- The Wizard and I: 맑고 희망적 톤, 또렷한 발음.\n- I’m Not That Girl / As Long as You’re M

### 7-2. PII(개인식별정보) 처리
- 사용자가 잘못해서 자신의 개인정보를 입력하더라도, 그 내용이 사용자 동의 없이 그대로 우리 DB에 저장되어서는 안 되겠죠.
- PII를 자동으로 감지하고 정보의 특성에 맞게 처리해봅시다.

In [ ]:
# 미들웨어에서 PII를 처리하기 위한 모듈입니다.
from langchain.agents.middleware import PIIMiddleware

# 사용할 middleware 객체를 미리 정의합니다. 세 개의 PII를 각각 처리해봅시다.
middleware_email = PIIMiddleware(
    pii_type='email', # 이메일
    detector=r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", # 정규표현식으로 이메일 패턴을 정의합니다.
    strategy='redact', # PII 삭제
    apply_to_input=True, # 입력값에 대해 PII 처리 로직 발동
)

middleware_credit_card = PIIMiddleware(
    pii_type='credit_card', # 신용카드 번호
    strategy='mask', # 마스킹 처리
    apply_to_input=True,
)

middleware_api_key = PIIMiddleware(
    pii_type='api_key', # API 키
    detector=r"sk-[a-zA-Z0-9]{32}", # OpenAI API key의 패턴입니다.
    strategy='block', # 오류 발생
)

# agent 생성
agent = create_agent(
    model=model,
    middleware=[middleware_email, middleware_credit_card, middleware_api_key], # 위에서 정의한 세 개의 PII 처리 middleware를 전달합니다.
)

In [126]:
# email 정보를 포함하여 agent를 호출해봅시다.
messages = [
    HumanMessage(content="AWS 요금이 4백만 원 청구됐어. 깎아달라고 불쌍하게 비는 메일좀 써줘. 내 AWS 계정은 sapy123@gmail.com이야."),
]

response = agent.invoke(
    {"messages":messages},
    )

pprint(response)

{'messages': [HumanMessage(content='AWS 요금이 4백만 원 청구됐어. 깎아달라고 불쌍하게 비는 메일좀 써줘. 내 AWS 계정은 [REDACTED_EMAIL]이야.', additional_kwargs={}, response_metadata={}, id='39b803b6-2ab0-4132-9199-7c6659984388'),
              AIMessage(content='아래는 AWS에 청구 감면(또는 크레딧 요청)을 정중하고 간절하게 요청하는 메일 템플릿입니다. 계정 정보, 청구서 번호, 사용내역 등 실제 정보를 괄호 안에 채워서 보내세요.\n\n제목:\n긴급 도움 요청 — 예상치 못한 AWS 요금 4,000,000원 청구 관련 감면/크레딧 요청 (계정: [REDACTED_EMAIL])\n\n본문:\n안녕하세요, AWS 고객지원팀 귀하\n\n저는 [이름/회사명]이며, AWS 계정([REDACTED_EMAIL])으로 서비스를 이용 중입니다. 이번 달 청구서에서 예상치 못하게 4,000,000원이 청구되어 매우 당황하고 있습니다. 저는 현재 개인(또는 소규모 사업자/학생 등)으로서 이 비용을 감당하기 어려운 상황입니다.\n\n상세 내용:\n- 청구 금액: 약 4,000,000원\n- 청구서/인보이스 번호: [인보이스 번호 또는 결제 내역 ID]\n- 청구일/기간: [청구 기간]\n- 의심되는 서비스/리전: [예: EC2 서울 리전, S3 등 — 가능하면 구체적으로 기재]\n- 특이사항: (예: 사용량 급증을 미처 인지하지 못했음 / 계정 일부가 악용된 의심 / 무료 체험 종료 후 자동 과금 등)\n\n저는 이번 청구가 저의 경제적 상황에 큰 부담이 되어 감면이나 크레딧을 간곡히 요청드립니다. 만약 부당한 사용(계정 탈취 등)으로 인한 것이라면 조사에 적극 협조하겠습니다. 먼저 다음 조치를 취해 두었습니다.\n- 현재 사용 중인 리소스(인스턴스/서비스)를 중지/종료함\n- 루트 계정 및 IAM 비밀번호 변경, 액세스 키 비활

In [127]:
#  신용카드 번호를 포함하여 agent를 호출해봅시다.
messages = [
    HumanMessage(content="환불 요청 메일좀 써줘. 내 결제 정보야.\nname: Jerry\nproduct: youtube premium\nprice: $8.80\ncredit card: 4239-1282-8367-7743"),
]

response = agent.invoke(
    {"messages":messages},
    )

pprint(response)

{'messages': [HumanMessage(content='환불 요청 메일좀 써줘. 내 결제 정보야.\nname: Jerry\nproduct: youtube premium\nprice: $8.80\ncredit card: 4239-1282-8367-7743', additional_kwargs={}, response_metadata={}, id='f8526f13-36be-4411-b31f-0706b81a34ca'),
              AIMessage(content='죄송하지만 보안상 전체 신용카드 번호(예: 4239-1282-8367-7743)는 이메일 내용에 포함해 드릴 수 없습니다. 대신 카드 번호를 제외한 형태로 환불 요청 메일 예시를 아래에 준비했습니다. 원하시면 카드 끝 4자리(7743), 거래일자, 거래ID(영수증 번호) 등 민감도가 낮은 정보만 추가해 드리겠습니다.\n\n제목: YouTube Premium 환불 요청 (결제 금액 $8.80)\n\n본문:\n안녕하세요, YouTube 고객지원팀 담당자님께\n\n제 이름은 Jerry입니다. 아래 결제 건에 대해 환불을 요청드립니다.\n\n- 상품: YouTube Premium\n- 결제 금액: $8.80\n- 결제일자: [결제일자 입력]\n- 결제 계정(이메일): [구독에 사용한 이메일 주소 입력]\n- 결제수단: 신용카드(끝 4자리: 7743) 또는 [결제수단 정보 입력]\n- 거래ID/영수증 번호: [있다면 입력]\n\n환불 사유: [예: 실수로 중복 결제됨 / 원치 않는 결제 발생 / 기타 사유 간단히 기재]\n\n환불은 원래 결제에 사용된 수단으로 처리해 주시길 요청드립니다. 환불 처리 가능 여부 및 필요하신 추가 정보(영수증 사본, 거래ID 등)를 안내해 주시면 감사하겠습니다.\n\n연락처: [전화번호 또는 대체 이메일 입력]\n\n감사합니다.\nJerry 드림\n\n참고: 이메일로 전체 카드번호를 보내지 마시고, 필요 시 고객지원 포털의 안전한 입력란이나 고객센터 전화로 제공하시

In [ ]:
# api key를 포함하여 agent를 호출해봅시다.
messages = [
    HumanMessage(content="내가 env 변수명 잘못 설정했나??\n OPENAPI_API_KEY=sk-proj4a2fcd5a1bdbd09e4fdd020d07b6a4380c84fdb07be5f674c5fa8e39b35e"),
]

''' 아래 코드는 실행시 PIIDetectionError가 발생합니다!
PIIDetectionError: Detected 1 instance(s) of api_key in text content
During task with name 'PIIMiddleware[api_key].before_model' and id '95c50dad-870c-5ce6-6178-0d9b216c4341'
'''
# response = agent.invoke(
#     {"messages":messages},
#     )

# pprint(response)

PIIDetectionError: Detected 1 instance(s) of api_key in text content